In [57]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import sys
from copy import deepcopy
import imodels
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import ElasticNet, LogisticRegression, Ridge, Lasso, RidgeCV, ElasticNetCV, LinearRegression, LassoCV
from collections import defaultdict
import guidance
import linearbuddy.data
import linearbuddy.prompts

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Set up a linear regression problem
Groundtruth is fit to the entire dataset (selected via CV) whereas the other models are fit to a small subset

In [58]:
# Set up data
# DSETS_CLASSIFICATION = ['pima_diabetes']
# X, y, feature_names = imodels.get_clean_dataset("pima_diabetes")

# california
# X, y, feature_names = imodels.get_clean_dataset("california_housing")
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, random_state=42, test_size=0.99
# )
# X = sklearn.preprocessing.StandardScaler().fit_transform(X)
# y = sklearn.preprocessing.StandardScaler().fit_transform(y.reshape(-1, 1)).flatten()

# iai
X, y, feats_raw, feats_abbrev = linearbuddy.data.get_iai_data()
feature_names = feats_abbrev.map(
    linearbuddy.data.ABBREV_TO_CLEAN_IAI.get).values.tolist()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.6
)

# preprocess split data
trans = sklearn.preprocessing.StandardScaler()
X_train = trans.fit_transform(X_train)
X_test = trans.transform(X_test)
transy = sklearn.preprocessing.StandardScaler()
y_train = transy.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = transy.transform(y_test.reshape(-1, 1)).flatten()
print("shapes", X.shape, y.shape, "nunique",
      np.unique(y).size, '-> train', X_train.shape)

shapes (12044, 14) (12044,) nunique 2 -> train (4817, 14)


In [60]:
def fit_and_get_feats(X_train, y_train, topk=2):
    PARAM_GRID_LINEAR_REGRESSION = [
        {
            "est": [
                RidgeCV(), ElasticNetCV(), LinearRegression(), LassoCV()
            ],
        },
    ]
    m = imodels.AutoInterpretableRegressor(
        # m = imodels.AutoInterpretableClassifier(
        param_grid=PARAM_GRID_LINEAR_REGRESSION, refit=True)
    m.fit(X_train, y_train, cv=3)

    # print("best params", m.est_.best_params_)
    # print("best score", m.est_.best_score_)
    # print("best estimator", m.est_.best_estimator_)
    # print("best estimator params", m.est_.best_estimator_.get_params())
    # print('selected from', m.param_grid)
    df = pd.DataFrame(m.est_.cv_results_).sort_values(
        "rank_test_score").reset_index()
    first_cols = ["rank_test_score", "mean_test_score", "std_test_score"]
    df = df[first_cols +
            [c for c in df.columns if c not in first_cols]].round(3)
    # remove std_ cols
    df = df[[c for c in df.columns if "std_" not in c]]

    # Refit top models with best params
    d = defaultdict(list)
    for i in range(topk):
        params = df.loc[i, 'params']
        clf = m.est_.best_estimator_.set_params(**params)
        clf.fit(X_train, y_train)
        clf = clf.steps[0][1]
        d['model'].append(deepcopy(clf))
        d['train_score'].append(clf.score(X_train, y_train))
        d['test_score'].append(clf.score(X_test, y_test))
        d['coef'].append(clf.coef_)
        d['intercept'].append(clf.intercept_)
    d = pd.DataFrame(d)
    return d


d_gt = fit_and_get_feats(X, y, topk=1)
d_small = fit_and_get_feats(X_train, y_train, topk=2)
d = pd.concat((d_gt, d_small))

### Interpretation
Note: these our coefficients after standardizing the inputs.

In [61]:
out = {'feature_names': feature_names + ['Intercept']}
for i in range(len(d)):
    coef = d.iloc[i].coef.tolist() + \
        [d.iloc[i].intercept.tolist()]
    if i == 0:
        out['GT'] = coef
    else:
        # out[f'{str(d.iloc[i]["model"])[:-4]} ({i})'] = coef
        out[i] = coef

coefs = pd.DataFrame.from_dict(out)
col1 = coefs.columns[1]
vabs = np.max(np.abs(coefs[col1]))

display(d.round(3).drop(
    columns=['coef', 'intercept']))
display(
    coefs
    .sort_values(by=col1)
    .style.background_gradient(
        cmap=sns.diverging_palette(
            20, 220, as_cmap=True, center='dark'),
        vmin=-vabs, vmax=vabs
    )
    .format(precision=2)
)

,model,train_score,test_score
0,RidgeCV(),0.113,0.110
0,RidgeCV(),0.108,0.106
1,LinearRegression(),0.108,0.106


,feature_names,GT,1,2
7,Full GCS score,-0.07,-0.13,-0.13
11,Thoracic tenderness,-0.02,-0.04,-0.04
10,Seatbelt sign,-0.00,0.04,0.04
3,Age,-0.00,-0.02,-0.02
9,Involvement in motor vehicle collision,0.01,0.03,0.03
6,Distracting pain,0.02,0.01,0.01
13,Vomiting,0.03,0.02,0.02
12,Thoracic trauma,0.05,0.08,0.08
4,Costal margin tenderness,0.06,0.10,0.10
2,Abdominal pain,0.06,0.12,0.12


# Let's ask GPT some questions about the models

In [74]:
# connect to a chat model like GPT-4 or Vicuna
gpt4 = guidance.llms.OpenAI("gpt-4-0613")
# gpt4 = guidance.llms.OpenAI("gpt-4-0314")
# gpt4 = guidance.llms.OpenAI("gpt-3.5-turbo")
program = guidance(linearbuddy.prompts.expl_program, llm=gpt4)

# gt_vs_subsampled models
# coef1 = coefs.loc[:, 'GT'].round(2).tolist()
# coef2 = coefs.loc[:, 1].round(2).tolist()

# 2_subsampled_models
# coef1 = coefs.loc[:, 1].round(2).tolist()
# coef2 = coefs.loc[:, 2].round(2).tolist()

# flipped_coefs - for the sake of illustraton, let's flip a couple of coefficients
# coef1 = coefs.loc[:, 'GT'].round(2).tolist()
# coef2 = deepcopy(coef1)
# coef2[0] = -coef2[0]
# coef2[-1] = -coef2[-1]

coef1 = coefs.loc[:, 'GT'].round(2).tolist()
coef2 = deepcopy(coef1)
idx_gcs = np.where(np.array(feature_names) == 'Full GCS score')[0][0]
coef2[idx_gcs] = -coef2[idx_gcs]

ans = program(
    # target='Housing prices in California',
    target='Risk of severe intra-abdominal injury',
    feature_list_str='\n- '.join(feature_names),
    coef1='\n- '.join([f'{f}: {c}' for f, c in zip(feature_names, coef1)]),
    coef2='\n- '.join([f'{f}: {c}' for f, c in zip(feature_names, coef2)]),
)

Stop program system You are a helpful and terse data-science assistant. user I am going to fit a linear model on a dataset that predicts Risk of severe intra-abdominal injury from the following features:
- Abdominal tenderness
- Abdominal wall trauma
- Abdominal pain
- Age
- Costal margin tenderness
- Decreased breath sounds
- Distracting pain
- Full GCS score
- Hypotension
- Involvement in motor vehicle collision
- Seatbelt sign
- Thoracic tenderness
- Thoracic trauma
- Vomiting 
How do you expect each of these features to affect the prediction? assistant Without specific data, it's hard to say definitively, but generally:

- Abdominal tenderness: Likely to increase risk, as it may indicate internal injury.
- Abdominal wall trauma: Likely to increase risk, as it suggests potential for internal damage.
- Abdominal pain: Likely to increase risk, as it may be a symptom of internal injury.
- Age: Could increase risk, as older individuals may be more susceptible to severe injury.
- Costal margin tenderness: Likely to increase risk, as it may indicate internal injury.
- Decreased breath sounds: Could increase risk, as it may suggest lung or chest injury.
- Distracting pain: Could increase or decrease risk, depending on the source of the pain.
- Full GCS score: Lower scores (indicating worse neurological function) would likely increase risk.
- Hypotension: Likely to increase risk, as it may be a sign of internal bleeding.
- Involvement in motor vehicle collision: Likely to increase risk, due to the potential for high-impact injury.
- Seatbelt sign: Likely to increase risk, as it suggests a high-impact injury.
- Thoracic tenderness: Likely to increase risk, as it may indicate chest or lung injury.
- Thoracic trauma: Likely to increase risk, as it suggests potential for internal damage.
- Vomiting: Could increase risk, as it may be a symptom of internal injury. user Here are the coefficients resulting from fitting the linear model after normalization (higher coefficients yield higher predicted Risk of severe intra-abdominal injury ):
- Abdominal tenderness: 0.21
- Abdominal wall trauma: 0.06
- Abdominal pain: 0.06
- Age: -0.0
- Costal margin tenderness: 0.06
- Decreased breath sounds: 0.12
- Distracting pain: 0.02
- Full GCS score: -0.07
- Hypotension: 0.11
- Involvement in motor vehicle collision: 0.01
- Seatbelt sign: -0.0
- Thoracic tenderness: -0.02
- Thoracic trauma: 0.05
- Vomiting: 0.03 
Explain what makes sense and what does not. assistant - Abdominal tenderness: Makes sense, as it's a direct symptom of potential internal injury.
- Abdominal wall trauma: Makes sense, though the coefficient is lower than expected, suggesting it's not as strong a predictor as abdominal tenderness.
- Abdominal pain: Makes sense, but like abdominal wall trauma, the coefficient is lower than expected.
- Age: It's surprising that age has no effect. Usually, older age is associated with higher risk.
- Costal margin tenderness: Makes sense, as it's a symptom of potential internal injury.
- Decreased breath sounds: Makes sense, as it could indicate lung or chest injury.
- Distracting pain: The low coefficient suggests it's not a strong predictor, which makes sense as it's not directly related to abdominal injury.
- Full GCS score: Negative coefficient makes sense, as lower GCS scores (worse neurological function) increase risk.
- Hypotension: Makes sense, as it could indicate internal bleeding.
- Involvement in motor vehicle collision: The low coefficient is surprising, as high-impact injuries are usually associated with higher risk.
- Seatbelt sign: Surprising that it has no effect, as it's usually associated with high-impact injuries.
- Thoracic tenderness: Surprising negative coefficient, as it's expected to increase risk.
- Thoracic trauma: Makes sense, but the coefficient is lower than expected.
- Vomiting: Makes sense, but the coefficient is lower than expected, suggesting it's not a strong predictor. user Here are the coefficients resulting from